<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/LangChain_ChatGithub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install --upgrade langchain deeplake openai tiktoken

In [1]:
import os

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

os.environ['OPENAI_API_KEY'] = 'sk-'
os.environ['ACTIVELOOP_TOKEN'] = '..'


In [2]:
embeddings = OpenAIEmbeddings(disallowed_special=())

In [5]:
!git clone https://github.com/chroma-core/chroma.git

^C


Cloning into 'chroma'...
fatal: not a git repository: 'C:/Users/Dell/studio/python/auto-trade/examples/chatgpt/chroma/.git'
fatal: remote did not send all necessary objects


In [6]:
import os
from langchain.document_loaders import TextLoader

root_dir = './tmp/chroma'
docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        try: 
            loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
            docs.extend(loader.load_and_split())
        except Exception as e: 
            pass

In [7]:
len(docs)

425

In [8]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)


Created a chunk of size 1030, which is longer than the specified 1000
Created a chunk of size 1103, which is longer than the specified 1000
Created a chunk of size 1020, which is longer than the specified 1000
Created a chunk of size 1540, which is longer than the specified 1000
Created a chunk of size 2528, which is longer than the specified 1000
Created a chunk of size 1620, which is longer than the specified 1000
Created a chunk of size 1710, which is longer than the specified 1000
Created a chunk of size 2982, which is longer than the specified 1000
Created a chunk of size 1129, which is longer than the specified 1000
Created a chunk of size 1628, which is longer than the specified 1000
Created a chunk of size 1041, which is longer than the specified 1000
Created a chunk of size 1030, which is longer than the specified 1000
Created a chunk of size 1103, which is longer than the specified 1000
Created a chunk of size 1035, which is longer than the specified 1000
Created a chunk of s

In [9]:
len(texts)

1074

In [10]:
username = "alphazz"
db = DeepLake(dataset_path=f"hub://{username}/chroma_source2", embedding_function=embeddings, public=True)
db.add_documents(texts)

Your Deep Lake dataset has been successfully created!


/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/alphazz/chroma_source2


hub://alphazz/chroma_source2 loaded successfully.


Evaluating ingest: 100%|██████████| 2/2 [01:04<00:00
-

Dataset(path='hub://alphazz/chroma_source2', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype      shape       dtype  compression
  -------   -------    -------     -------  ------- 
 embedding  generic  (1074, 1536)  float32   None   
    ids      text     (1074, 1)      str     None   
 metadata    json     (1074, 1)      str     None   
   text      text     (1074, 1)      str     None   


['d43bb0d9-ee0f-11ed-8af7-8c554ae85ff9',
 'd43bb0da-ee0f-11ed-bdec-8c554ae85ff9',
 'd43bb0db-ee0f-11ed-97d6-8c554ae85ff9',
 'd43bb0dc-ee0f-11ed-8b82-8c554ae85ff9',
 'd43bb0dd-ee0f-11ed-ace4-8c554ae85ff9',
 'd43bb0de-ee0f-11ed-a1c9-8c554ae85ff9',
 'd43bb0df-ee0f-11ed-8260-8c554ae85ff9',
 'd43bb0e0-ee0f-11ed-90e8-8c554ae85ff9',
 'd43bb0e1-ee0f-11ed-9fee-8c554ae85ff9',
 'd43bb0e2-ee0f-11ed-89c3-8c554ae85ff9',
 'd43bb0e3-ee0f-11ed-b64e-8c554ae85ff9',
 'd43bb0e4-ee0f-11ed-bc90-8c554ae85ff9',
 'd43bb0e5-ee0f-11ed-ad15-8c554ae85ff9',
 'd43bb0e6-ee0f-11ed-b03d-8c554ae85ff9',
 'd43bb0e7-ee0f-11ed-80be-8c554ae85ff9',
 'd43bb0e8-ee0f-11ed-b8e8-8c554ae85ff9',
 'd43bb0e9-ee0f-11ed-8f2e-8c554ae85ff9',
 'd43bb0ea-ee0f-11ed-859f-8c554ae85ff9',
 'd43bb0eb-ee0f-11ed-b82b-8c554ae85ff9',
 'd43bb0ec-ee0f-11ed-b447-8c554ae85ff9',
 'd43bb0ed-ee0f-11ed-a4dd-8c554ae85ff9',
 'd43bb0ee-ee0f-11ed-86cd-8c554ae85ff9',
 'd43bb0ef-ee0f-11ed-b5b3-8c554ae85ff9',
 'd43bb0f0-ee0f-11ed-a2fd-8c554ae85ff9',
 'd43bb0f1-ee0f-

In [11]:
db = DeepLake(dataset_path=f"hub://{username}/chroma_source2", read_only=True, embedding_function=embeddings)

|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/alphazz/chroma_source2



|

hub://alphazz/chroma_source2 loaded successfully.

Deep Lake Dataset in hub://alphazz/chroma_source2 already exists, loading from the storage
Dataset(path='hub://alphazz/chroma_source2', read_only=True, tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype      shape       dtype  compression
  -------   -------    -------     -------  ------- 
 embedding  generic  (1074, 1536)  float32   None   
    ids      text     (1074, 1)      str     None   
 metadata    json     (1074, 1)      str     None   
   text      text     (1074, 1)      str     None   


In [12]:
retriever = db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['fetch_k'] = 100
retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 10


In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model='gpt-3.5-turbo')
qa = ConversationalRetrievalChain.from_llm(model,retriever=retriever)

In [14]:
questions = [
    "What does Chroma do?",
    "How to use Chroma?"
] 
chat_history = []

for question in questions:  
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result['answer']))
    print(f"Question:\n {question} \n")
    print(f"Answer:\n {result['answer']} \n\n")

Question:
 What does Chroma do? 

Answer:
 Chroma is an open-source embedding database that makes it easy to build LLM (Language Learning Model) apps by making knowledge, facts, and skills pluggable for LLMs. It is a vector database that stores embeddings and allows you to search by nearest neighbors rather than by substrings like a traditional database. By default, Chroma uses Sentence Transformers to embed for you but you can also use OpenAI embeddings, Cohere (multilingual) embeddings, or your own. 


Question:
 How to use Chroma? 

Answer:
 Sure! Chroma is a package that gives you a JS/TS interface to talk to a backend Chroma DB over REST. Here are some general steps to get started:

1. Chroma needs to be running in order for this client to talk to it. Please see the [🧪 Usage Guide](https://docs.trychroma.com/usage-guide) to learn how to quickly set this up.

2. Install Chroma by running `pip install chromadb` for the python client, `npm install chromadb` for JavaScript.

3. Use th

In [15]:
def ask(question, chat_history):
  response = qa({"question": question, "chat_history": chat_history})
  print(f"Question:\n {question}\n")
  print(f"Answer:\n {response['answer']}\n")

In [16]:
ask("What's the main programming language used in Chroma?", chat_history)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-BIyBdUaZsiVZDpsR7VJtHIeH on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..


Question:
 What's the main programming language used in Chroma?

Answer:
 Chroma supports two programming languages: Python and JavaScript. You can use `pip install chromadb` to install the Python client or `npm install chromadb` to install the JavaScript client.



In [17]:
ask('Summarize the storage part of Chroma', chat_history)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-BIyBdUaZsiVZDpsR7VJtHIeH on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-BIyBdUaZsiVZDpsR7VJtHIeH on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

Question:
 Summarize the storage part of Chroma

Answer:
 Chroma is an open-source embedding database that allows you to store embeddings and search by nearest neighbors. It uses vector databases to store embeddings and provides a JS/TS interface to talk to a backend Chroma database over REST. By default, Chroma uses Sentence Transformers to embed, but you can also use OpenAI embeddings, Cohere (multilingual) embeddings or your own. It also has a client-server mode where you can docker-compose up to build. The core API has only four functions, which are createCollection, getCollection, getOrCreateCollection, and modify. Chroma is licensed under Apache 2.0.



In [18]:
ask('Tell me more about Sentence Transformers', chat_history)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-BIyBdUaZsiVZDpsR7VJtHIeH on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-BIyBdUaZsiVZDpsR7VJtHIeH on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

Question:
 Tell me more about Sentence Transformers

Answer:
 Sentence Transformers are a type of pre-trained model that maps sentences to fixed-length vectors. Chroma by default uses Sentence Transformers to create embeddings for documents stored in its vector database, which allows you to search by nearest neighbors.



In [19]:
ask('Show me some example code on how to use Chroma to store embeddings', chat_history)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-BIyBdUaZsiVZDpsR7VJtHIeH on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-BIyBdUaZsiVZDpsR7VJtHIeH on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

Question:
 Show me some example code on how to use Chroma to store embeddings

Answer:
 Yes, here's an example code on how to use Chroma to store embeddings:

```python
import chromadb

# create a new Chroma client
client = chromadb.Client()

# create a new collection
collection = client.create_collection("my_collection")

# add documents to the collection
collection.add(
    documents=["This is document1", "This is document2"],
    metadatas=[{"source": "notion"}, {"source": "google-docs"}],
    ids=["doc1", "doc2"],
)

# query the collection for similar documents
results = collection.query(
    query_texts=["This is a query document"],
    n_results=2,
    where={"metadata_field": "is_equal_to_this"},
    where_document={"$contains":"search_string"},
)
```

In this example, we first create a new Chroma client using the `chromadb.Client()` function. We then create a new collection using the `client.create_collection()` method, which takes in the name of the collection as an argument. 

In [ ]:
ask('What is the Python class for Chroma query interface?', chat_history)

In [ ]:
ask('Show me the public functions of class Client', chat_history)

In [ ]:
ask('What are the underlying databases used by Chroma?', chat_history)

In [ ]:
ask('Which class implements the DuckDB support?', chat_history)